In [1]:
import pandas as pd
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('../data/data.csv')
data.head(10)

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1
5,2021-09-27 00:06:50.000000,5,5,2377014068362699676,0,2,2,5,0.004,0.337634,-3.222757,37,0.004,0.338195,-3.221755,1,1
6,2021-09-27 00:07:34.000000,6,6,6863358899511896876,0,3,0,6,0.002,0.033805,-3.063872,29,0.002,0.037688,-3.111623,1,1
7,2021-09-27 00:08:49.000000,7,7,2876502170484631685,0,4,1,7,0.010,0.026041,-2.509060,11464231,0.450,0.020563,-2.753571,1,1
8,2021-09-27 00:09:08.000000,8,8,5839858970958967275,0,4,3,8,0.020,0.033933,-3.888843,243,0.020,0.031670,-3.923608,1,1
9,2021-09-27 00:09:16.000000,1,9,4868455078459394303,0,4,4,9,0.010,0.079909,-2.997711,11464232,0.670,0.022754,-1.840219,1,1


In [ ]:
# удалим признаки, которые не нужны по условию
# нам известно ограничение на revard, поэтому можно удалить rate0 и rate1, потому что по условию они равны 1
columns_to_drop = ['campaign_clicks', 'rate0', 'rate1']
data = data.drop(columns=columns_to_drop)

data['date_time'] = pd.to_datetime(data['date_time'])
data.head()

In [ ]:
def analysis(data: pd.DataFrame):
    # проверим вначале на наличие NaN-значений
    null_values = data.isnull().sum().sum()
    print(f"Count of NaN {null_values}")
    
    # посмотрим краткое описание данных, заметим, что impressions бесполезен, потому что константа
    print(data.describe())
    print()
    
    # посмотрим на временной интервал: 
    print(data['date_time'].dt.date.value_counts())
    print()
    # Заметим, что 2021-09-01 скорее всего лишний, потому что он в единственном экземпляре и выбивается из интервала
    
    # Посмотрим на категориальные фичи, непопулярные будем объединять в одну группу, совсем редкие будем удалять
    print(f"Unique {data['zone_id'].nunique()}")
    print(data['zone_id'].value_counts())
    print()
    # некоторых зон совсем мало, можно почистить
    
    print(f"Unique {data['banner_id'].nunique()}")
    print(data['banner_id'].value_counts())
    print()
    # некоторых баннеров совсем мало, можно почистить
    
    print(f"Unique {data['os_id'].nunique()}")
    print(data['os_id'].value_counts())
    print()
    # с 7 по 10 можно объединить
    
    print(f"Unique {data['country_id'].nunique()}")
    print(data['country_id'].value_counts())
    print()
    # со странами все хорошо
    

In [ ]:
analysis(data)

Необходима дополнительная чистка, потому что в появились новые NaN значения, так же уберем столбец с impressions, oaid_hash уберем, потому что не использовался в первой модели из домашки и сделаем простую фильтрацию по аналогии с тем, как делали раньше. Добавим часы 

In [ ]:
data = data.drop(columns=["impressions", 'oaid_hash'])
data = data[data['date_time'] > '2021-09-02']
data['hour'] = data['date_time'].dt.hour
data.head()

Удалим строки, где нет значений

In [ ]:
print(data.shape)
data = data.dropna()
print(data.shape)

Делим на train/test по последнему дню

In [ ]:
train_data = data[data['date_time'] < '2021-10-02']
test_data = data[data['date_time'] >= '2021-10-02']

In [ ]:
train_data.shape, test_data.shape

In [ ]:
categorical_features = ["zone_id", "banner_id", "os_id", "country_id", "hour"]

Фильтруем по условию, что banner_id0 = banner_id

In [ ]:
test_data = test_data[test_data['banner_id0'] == test_data['banner_id']]
test_copy = test_data.copy()

y_test = test_data['clicks']
X_test = test_data.drop(['date_time', 'clicks'], axis=1)

y_train = train_data['clicks']
X_train = train_data.drop(['date_time', 'clicks'], axis=1)

# policy 1
X_test_1 = X_test.copy()
X_test_1['banner_id'] = X_test_1['banner_id1']

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(X_train[categorical_features])
X_test = ohe.transform(X_test[categorical_features])
X_test_1 = ohe.transform(X_test_1[categorical_features])

In [ ]:
from sklearn.linear_model import LogisticRegression

# Используем логистическую регрессию с 'liblinear', упомянутую на лекции, он не является SGD, 
# так же взяли l2 регуляризацию
def create_model(C=0.01):
    return LogisticRegression(solver='liblinear', C=C, penalty="l2")

Используем модель из первой домашки

In [ ]:
# Сравниваем 2 модели, получаем, что наша стала лучше базового решения
from sklearn.metrics import roc_auc_score, log_loss

model = create_model(C=0.01).fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc_metric = roc_auc_score(y_test, y_pred_proba)
log_loss_metric = log_loss(y_test, y_pred_proba)
print(f"roc_auc = {roc_auc_metric}, log_loss = {log_loss_metric}")

Посчитаем CIPS
Нужно сравнить 2 случайные величины или по-другому: оценить вероятность того, что одна из них больше другой
Обозначим за $N_0$ результат 1 баннера (нормальной с в), за $N_1$ второго

$P(N_0 > N_1) = P(N_0 - N_1 > 0) = 1 - P(N_0 - N_1 <= 0) = 1 - F_{N_0 - N_1}(0)$

Это и будет нашей функцией для подсчета policy

In [ ]:
import numpy as np
from scipy.stats import norm


In [ ]:
def calc_policy(coeff_sum0, g0, coeff_sum1, g1):
    return 1. - norm.cdf(0., loc=coeff_sum0 - coeff_sum1, scale=np.sqrt(g0 ** 2 + g1 ** 2) + 1e-6)


In [ ]:
pi0 = calc_policy(test_copy['coeff_sum0'], test_copy['g0'], test_copy['coeff_sum1'], test_copy['g1'])

Посчитаем новые коэффициенты так как указано в условии

In [ ]:
from scipy.special import logit

test_copy['coeff_sum0_new'] = logit(model.predict_proba(X_test)[:, 1])
test_copy['coeff_sum1_new'] = logit(model.predict_proba(X_test_1)[:, 1])

pi1 = calc_policy(test_copy['coeff_sum0_new'], test_copy['g0'], test_copy['coeff_sum1_new'], test_copy['g1'])

Берем lambda = 10

In [ ]:
l = 10

In [ ]:
cips = np.mean(y_test * np.minimum(pi1 / (pi0 + 1e-10), l))
print(cips)


Посчитаем новые коэффициенты так как указано в условии

In [18]:
from scipy.special import logit

test_copy['coeff_sum0_new'] = logit(model.predict_proba(X_test)[:, 1])
test_copy['coeff_sum1_new'] = logit(model.predict_proba(X_test_1)[:, 1])

pi1 = calc_policy(test_copy['coeff_sum0_new'], test_copy['g0'], test_copy['coeff_sum1_new'], test_copy['g1'])

Берем lambda = 10

In [19]:
l = 10

In [20]:
cips = np.mean(y_test * np.minimum(pi1 / (pi0 + 1e-10), l))
print(cips)


0.06751013434965179
